# Определение стоимости автомобилей

## Подготовка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.dummy import DummyRegressor
from catboost import CatBoostRegressor
from catboost import cv, Pool
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error,make_scorer
import lightgbm as lgb
import time

In [2]:
df = pd.read_csv('...')
df.head(5)

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


При просмотре данных видим, что есть пустоты, аномальные значения, строки с датами.

Дополнительно,исправим наименования столбцов в змеиный регистр.

In [3]:
df.columns = df.columns.str.lower()#переводим наименование столбцов в нижний регистр
df.rename(columns = {
    'datecrawled':'date_crawled', 'vehicletype':'vehicle_type', 'registrationyear':'registration_year',
    'registrationmonth':'registration_month', 'fueltype':'fuel_type',
       'notrepaired':'not_repaired', 'datecreated':'date_created', 
    'numberofpictures':'number_of_pictures', 'postalcode':'postal_code','lastseen':'last_seen'
    },inplace=True)# переименование в змеиный регистр
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   date_crawled        354369 non-null  object
 1   price               354369 non-null  int64 
 2   vehicle_type        316879 non-null  object
 3   registration_year   354369 non-null  int64 
 4   gearbox             334536 non-null  object
 5   power               354369 non-null  int64 
 6   model               334664 non-null  object
 7   kilometer           354369 non-null  int64 
 8   registration_month  354369 non-null  int64 
 9   fuel_type           321474 non-null  object
 10  brand               354369 non-null  object
 11  not_repaired        283215 non-null  object
 12  date_created        354369 non-null  object
 13  number_of_pictures  354369 non-null  int64 
 14  postal_code         354369 non-null  int64 
 15  last_seen           354369 non-null  object
dtypes:

In [4]:
#рассмотрим численные признаки
df.describe()

,price,registration_year,power,kilometer,registration_month,number_of_pictures,postal_code
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


In [5]:
df.describe(include='object')#рассмотрим категориальные признаки.

,date_crawled,vehicle_type,gearbox,model,fuel_type,brand,not_repaired,date_created,last_seen
count,354369,316879,334536,334664,321474,354369,283215,354369,354369
unique,271174,8,2,250,7,40,2,109,179150
top,2016-03-24 14:49:47,sedan,manual,golf,petrol,volkswagen,no,2016-04-03 00:00:00,2016-04-06 13:45:54
freq,7,91457,268251,29232,216352,77013,247161,13719,17


### Работа с аномальными значениями

Столбец `registration_year` имеет аномальные значения года. Возьмем года с 1900 по 2016,остальное будем считать выбросами.

Столбец `number_of_pictures` содержит только 0, его можно удалить, так как на предсказание модели он не повлияет. 

В столбце `registration_month` содержится нулевые значения, однако при их удалении,могут пострадать более важные данные,поэтому трогать данный столбец не будем.

В поле `power` имеются нулевые значения,а так же аномально-высокие значения мощности. На 2016 год самый мощный сирийный двигатель имеет 500 лошадиных сил. Возьмем этот порог за оптимальную мощность.

In [6]:
df = df[df['power'] != 0]
df = df[df['power'] < 500]
df.shape

(313646, 16)

In [7]:
df = df.drop('number_of_pictures',axis=1)

In [8]:
df['date_crawled'].max()

'2016-04-07 14:36:58'

In [9]:
df = df[(df['registration_year']>1900)]
df = df[(df['registration_year']<=2016)]
df.shape 

(302864, 15)

In [10]:
df.describe()# проверим исправления

,price,registration_year,power,kilometer,registration_month,postal_code
count,302864.000000,302864.000000,302864.000000,302864.000000,302864.000000,302864.000000
mean,4749.757799,2002.749336,120.347522,128351.768451,5.964238,51116.465744
std,4613.138811,6.597520,53.440986,36859.289952,3.599468,25810.631378
min,0.000000,1910.000000,1.000000,5000.000000,0.000000,1067.000000
25%,1250.000000,1999.000000,75.000000,125000.000000,3.000000,30880.000000
50%,3000.000000,2003.000000,110.000000,150000.000000,6.000000,50169.000000
75%,6900.000000,2007.000000,150.000000,150000.000000,9.000000,71711.000000
max,20000.000000,2016.000000,490.000000,150000.000000,12.000000,99998.000000


Видим прямую зависимость цены от мощности автомобиля, чем мощнее авто,тем дороже его стоимость.

Прямая корреляция цены наблюдается и с годом регистрации - чем новее автомобиль,тем дороже цена.

Присутствует обратная корреляция цены с показателем пробега: чем больше пробег, тем дешевле авто.

Автомобили с автоматической коробкой передач ценятся дороже, чем автомобили с механической.


In [12]:
#удалим столбцы с датами и уберем строки, где цена равна 0.
df = df.drop(['date_crawled','date_created','last_seen'],axis=1)
df = df[df['price'] != 0]

In [13]:
df.isna().mean() # доля пустот в данных

price                 0.000000
vehicle_type          0.037235
registration_year     0.000000
gearbox               0.018150
power                 0.000000
model                 0.037747
kilometer             0.000000
registration_month    0.000000
fuel_type             0.052491
brand                 0.000000
not_repaired          0.144820
postal_code           0.000000
dtype: float64

### Работа с пропусками

В столбце `vehicle_type` и `model`  заполнить пустоты конкретными данными не удастся, поэтому их либо удаляем, либо заполним заглушкой "no_info". Я выберу последнее.

В столбцах `gearbox` и  `fuel_type` можем произвести замену пустот по принципу наиболее распространенных значений.
В `gearbox` это значение manual, в `fuel_type` это значение petrol.

В столбце `not_repaired` пустые значения будем принимать за "no", предполагая, что Nan значит,что машина не была разбита.

In [14]:
df['vehicle_type'] = df['vehicle_type'].fillna('no_info')
df['model'] = df['model'].fillna('no_info')

df['gearbox'] = df['gearbox'].fillna('manual')
df['fuel_type'] = df['fuel_type'].fillna('petrol')

df['not_repaired'] = df['not_repaired'].fillna('no')

## Анализ моделей

### Обучение модели LGBMRegressor

In [15]:
#признаки и таргет.
X = ['registration_year', 'power', 'kilometer', 'registration_month',
       'postal_code','vehicle_type', 'gearbox', 'model', 'fuel_type', 'brand',
       'not_repaired']
#категориальные признаки.
cat_features = ['vehicle_type', 'gearbox', 'model', 'fuel_type', 'brand',
       'not_repaired']

y = ['price']

In [16]:
df_lgb = df.copy()
df_lgb[cat_features] = df_lgb[cat_features].astype('category')

In [17]:
#обучающая и тестовая выборки
train, test = train_test_split(df_lgb, test_size=0.25, random_state=42)
print(train.shape, test.shape) #проверяем обьем выборок,деление произошло корректно

(222354, 12) (74118, 12)


In [18]:
%%time
#объявляем модель
model_lgb = lgb.LGBMRegressor(
    n_jobs=-1,
    random_state=123
)
params = {
    'max_depth': range(1,10),
    'num_leaves': [30, 31],
    'learning_rate': [0.01, 0.05, 0.1,0.2],
    'n_estimators': [20, 50, 100]
}
# функция расчета метрики RMSE
def rmse(y_true, y_predicted):
    return mean_squared_error(y_true, y_predicted, squared=False)  

# мера, используемая при кросс-валидации - RMSE
scoring = make_scorer(rmse, greater_is_better=False)

grid_search_lgb = GridSearchCV(
        estimator=model_lgb,
        param_grid=params,
        cv=5,
        scoring=scoring,
    )


grid_search_lgb.fit(train[X], train[y].values.ravel())

CPU times: user 1h 33min 28s, sys: 21.6 s, total: 1h 33min 50s
Wall time: 1h 34min 36s


GridSearchCV(cv=5, estimator=LGBMRegressor(random_state=123),
             param_grid={'learning_rate': [0.01, 0.05, 0.1, 0.2],
                         'max_depth': range(1, 10),
                         'n_estimators': [20, 50, 100],
                         'num_leaves': [30, 31]},
             scoring=make_scorer(rmse, greater_is_better=False))

In [19]:
grid_search_lgb.best_params_

{'learning_rate': 0.2, 'max_depth': 9, 'n_estimators': 100, 'num_leaves': 31}

In [20]:
grid_search_lgb.best_score_

-1593.8899769347593

In [21]:
start_time = time.time()


#обучаем модель на лучших параметрах 
model_lgb = lgb.LGBMRegressor(max_depth=9,
                              num_leaves=31,learning_rate=0.2,
                              n_estimators=100,n_jobs=-1,
                              random_state=123).fit(train[X],train[y])


t_lgb_f = (time.time() - start_time)

In [22]:
start_time = time.time()


# предсказываем target на Тестовой выборке
y_pred = model_lgb.predict(test[X])
# считаем метрику RMSE, должна быть не больше 2500
RMSE_lgb = mean_squared_error(test[y], y_pred,squared=False)
t_lgb_p = (time.time() - start_time)


RMSE_lgb

1587.6314924366

С помощью GridSearchCV из библиотеки sklearn произвели поиск параметров для модели LGBMRegressor. Поиск гиперпараметров производился порядка 1.5-2часа. 

У модели LGBMRegressor метрика качества RMSE на тестовой выборке получилась 1587, что меньше заданного в задаче порогового значения. Цель достигнута.

### Обучение модели CatBoostRegressor

In [23]:
#обучающая и тестовая выборки
train, test = train_test_split(df, test_size=0.25, random_state=42)
print(train.shape, test.shape) #проверяем обьем выборок,деление произошло корректно

(222354, 12) (74118, 12)


In [24]:
#фичи и таргеты
X = ['registration_year', 'power', 'kilometer', 'registration_month',
       'postal_code','vehicle_type', 'gearbox', 'model', 'fuel_type', 'brand',
       'not_repaired']

#категориальные признаки
cat_features = ['vehicle_type', 'gearbox', 'model', 'fuel_type', 'brand',
       'not_repaired']

y = ['price']

In [25]:
#формируем тренировочный и тестовый датасет для кэтбуста
train_data = Pool(data=train[X],
                  label=train[y],
                  cat_features=cat_features
                 )
test_data = Pool(data=test[X],
                  label=test[y],
                  cat_features=cat_features
                 )

In [26]:
#обозначим используемые параметры,которые будем подбирать кросс-валидацией.
parametres = {'cat_features': cat_features,
              'eval_metric': 'RMSE',
          'loss_function': 'RMSE',
              'random_seed': 42,
              'verbose':100}

In [27]:
# запускаем кросс-валидацию, исполнение порядка 25 мин.

cv_data = cv(
    params = parametres,
    pool = train_data,
    fold_count=5,
    shuffle=True,
    partition_random_seed=42,
    stratified=False,
    verbose=False,
    early_stopping_rounds=200
)

Training on fold [0/5]

bestTest = 1655.327846
bestIteration = 999

Training on fold [1/5]

bestTest = 1634.064329
bestIteration = 999

Training on fold [2/5]

bestTest = 1645.36919
bestIteration = 999

Training on fold [3/5]

bestTest = 1641.487912
bestIteration = 999

Training on fold [4/5]

bestTest = 1656.989479
bestIteration = 999



In [28]:
# посмотрим на получившийся датасет итераций и RMSE на каждом шагу 
cv_data.head(5)

,iterations,test-RMSE-mean,test-RMSE-std,train-RMSE-mean,train-RMSE-std
0,0,6521.461315,23.614537,6521.504003,6.335458
1,1,6358.875261,23.606253,6358.963894,6.022796
2,2,6200.816336,23.974962,6200.821118,5.401766
3,3,6050.301234,24.733154,6050.242955,4.509780
4,4,5902.734304,24.197323,5902.847712,4.914033


In [29]:
#минимальное значение метрики
rmse_train = cv_data['test-RMSE-mean'].min()
rmse_train

1646.6477511954756

In [31]:
start_time = time.time()

model = CatBoostRegressor(**parametres).fit(train_data)

t_cbr_f = (time.time() - start_time)

Learning rate set to 0.096164
0:	learn: 4321.8326813	total: 218ms	remaining: 3m 38s
100:	learn: 1746.2419941	total: 15.6s	remaining: 2m 18s
200:	learn: 1659.2439730	total: 31s	remaining: 2m 3s
300:	learn: 1617.5820215	total: 45.8s	remaining: 1m 46s
400:	learn: 1589.7370876	total: 1m	remaining: 1m 30s
500:	learn: 1569.9840435	total: 1m 15s	remaining: 1m 15s
600:	learn: 1553.0331086	total: 1m 31s	remaining: 1m
700:	learn: 1538.7987243	total: 1m 46s	remaining: 45.5s
800:	learn: 1525.7771455	total: 2m 2s	remaining: 30.4s
900:	learn: 1514.3210437	total: 2m 17s	remaining: 15.1s
999:	learn: 1504.2647574	total: 2m 32s	remaining: 0us


In [32]:
start_time = time.time()


predictions = model.predict(test_data)
RMSE_cat = mean_squared_error(test[y], predictions,squared=False)
t_cbr_p = (time.time() - start_time)

RMSE_cat

1574.9516797774497

С помощью встроенной кросс-валидации произвели поиск гиперпараметров для модели CatBoostRegressor, данный процесс занял порядка 25 минут.

Метрика качества RMSE  в данной модели CatBoostRegressor так же показала хороший результат, ее значение - 1574.

### Обучение моделей DummyRegressor и LinearRegression для сравнения результатов

In [33]:
df_ohe = pd.get_dummies(df,drop_first=True)#метод OHE + избегаем дамми-ловушку.
df_ohe.head()

,price,registration_year,power,kilometer,registration_month,postal_code,vehicle_type_convertible,vehicle_type_coupe,vehicle_type_no_info,vehicle_type_other,...,brand_skoda,brand_smart,brand_sonstige_autos,brand_subaru,brand_suzuki,brand_toyota,brand_trabant,brand_volkswagen,brand_volvo,not_repaired_yes
1,18300,2011,190,125000,5,66954,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
2,9800,2004,163,125000,8,90480,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1500,2001,75,150000,6,91074,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,3600,2008,69,90000,7,60437,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
5,650,1995,102,150000,10,33775,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [34]:
train, test = train_test_split(df_ohe, test_size=0.25, random_state=42)
print(train.shape, test.shape) #проверяем обьем выборок,деление произошло корректно

(222354, 310) (74118, 310)


In [35]:
train_features = train.drop('price',axis=1)
train_target = train['price']

test_features = test.drop('price',axis=1)
test_target = test['price']

In [36]:
start_time = time.time()

dummy_r = DummyRegressor(strategy="mean").fit(train_features, train_target)
t_dummy_f = (time.time() - start_time)

In [37]:
start_time = time.time()

pred = dummy_r.predict(test_features)
RMSE_dummy = mean_squared_error(test_target, pred,squared=False)
t_dummy_p = (time.time() - start_time)

RMSE_dummy

4611.487567525834

In [38]:
start_time = time.time()

lin_reg = LinearRegression().fit(train_features, train_target)

t_lin_f= (time.time() - start_time)

In [39]:
start_time = time.time()

pred_lin = lin_reg.predict(test_features)
RMSE_lin = mean_squared_error(test_target, pred_lin,squared=False)

t_lin_p = (time.time() - start_time)

RMSE_lin

2590.536218930414

# Выводы

Подведем итоги нашего эксперимента. Обратим внимание на временные ресурсы, затраченные при работе с моделями: 

In [40]:
table = pd.DataFrame(index=['Время обучения','Время предсказания','Качество предсказания,RMSE'],
                    data={
  "LightGBM": [t_lgb_f, t_lgb_p, RMSE_lgb],
  "CatBoostRegressor": [t_cbr_f, t_cbr_p, RMSE_cat],
  'linearRegression': [t_lin_f, t_lin_p, RMSE_lin],
  'DummyRegressor': [t_dummy_f, t_dummy_p, RMSE_dummy]
})
table

,LightGBM,CatBoostRegressor,linearRegression,DummyRegressor
Время обучения,4.865045,154.968789,20.038655,0.000902
Время предсказания,0.597336,0.376136,0.226436,0.001760
"Качество предсказания,RMSE",1587.631492,1574.951680,2590.536219,4611.487568


Мы видим, что наилучшим значением метрики RMSE было значние, полученное моделью CatBoost. Данная модель удабна тем, что к ее обучению не требуется переделывать или кодировать данные,  она сама умеет обрабатывать категориальные переменные в задачах регрессии.

Простые модели, такие как лин.регрессия или dummy-модель конечно могут предсказать данные, но значения RMSE будет значительно хуже, а так же требуется дополнительное кодирование категориальных переменных, чтобы рассчитать задачу регрессии.

Модель LightGBM тоже практически не требует подготовки данных для обучения, это безусловно,большой плюс. работает данная модель тоже быстро. Подбор гиперпараметров через GridSearch достаточно долго обрабатывался, но зато результат метрики на тестовой выборке отличный.